# Storing Data in PostGreSQL

In [46]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


from tqdm import tqdm
from nba_api.stats.static import players, teams

import sys

%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from src.data.make_team_dataset import load_team_data


from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import ElasticNetCV, LogisticRegression, SGDClassifier

from sklearn.pipeline import make_pipeline

import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
import xgboost as xgb
from sklearn.ensemble import StackingClassifier

from sklearn.neighbors import KNeighborsClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.feature_selection import VarianceThreshold, RFE, RFECV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import sklearn.model_selection

from sklearn.model_selection import learning_curve, validation_curve
from sklearn.metrics import confusion_matrix, f1_score
import pickle


from sklearn.model_selection import ParameterGrid
import parfit.parfit as pf

import plotly.express as px
import plotly.graph_objects as go

import optuna
import joblib
import psycopg2
import sqlite3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
pd.options.display.max_columns=100

In [10]:
def load_team_data(seasons):
    """Loads basic, advanced, and scoring boxscores from
    seasons 2000-2020 and merges them into one dataframe
    """


    basic_gls_all, adv_gls_all, scoring_gls_all = [], [], []
    for season in seasons:
        basic_gls = pd.read_csv('../data/team_boxscores_basic/team_boxscores_{}.csv'.format(season))
        basic_gls_all.append(basic_gls)

        adv_gls = pd.read_csv('../data/team_boxscores_advanced/team_advanced_boxscores_{}.csv'.format(season))
        adv_gls_all.append(adv_gls)

        scoring_gls = pd.read_csv('../data/team_boxscores_scoring/team_scoring_boxscores_{}.csv'.format(season))
        scoring_gls_all.append(scoring_gls)

    basic_gls_all_df = pd.concat(basic_gls_all)
    adv_gls_all_df = pd.concat(adv_gls_all)
    scoring_gls_all_df = pd.concat(scoring_gls_all)
    
    basic_gls_all_df = basic_gls_all_df.iloc[:, :30]

    temp = pd.merge(basic_gls_all_df, adv_gls_all_df, how='left', on=['GAME_ID', 'TEAM_ABBREVIATION'], suffixes=['', '_y'])
    full_df = pd.merge(temp, scoring_gls_all_df, how='left', on=['GAME_ID', 'TEAM_ABBREVIATION'], suffixes=['', '_y'])

    full_df = full_df.drop(columns = ['TEAM_ID_y', 
                                      'TEAM_NAME_y', 'TEAM_CITY',
                                      'TEAM_CITY_y', 'MIN_y'])
    return full_df

seasons = ['200{}-0{}'.format(x, x+1) if x != 9 else '2009-10' for x in range(0, 10)]
seasons2 = ['20{}-{}'.format(x, x+1) for x in range(10, 21)]
seasons.extend(seasons2)
print(seasons)
team_data = load_team_data(seasons)

['2000-01', '2001-02', '2002-03', '2003-04', '2004-05', '2005-06', '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21']


In [13]:
def clean_team_data(df):
    """This function cleans the team_data
    1) Changes W/L to 1/0 
    2) Changes franchise abbreviations to their most 
    recent abbreviation for consistency
    3) Converts GAME_DATE to datetime object
    4) Creates a binary column 'HOME_GAME'
    5) Removes 3 games where advanced stats were not collected
    """
    df = df.copy()
    df['WL'] = (df['WL'] == 'W').astype(int)
    
    abbr_mapping = {'NJN':'BKN',
                   'CHH':'CHA',
                   'VAN':'MEM',
                   'NOH':'NOP',
                   'NOK':'NOP',
                   'SEA':'OKC'}
    
    df['TEAM_ABBREVIATION'] = df['TEAM_ABBREVIATION'].replace(abbr_mapping)
    df['MATCHUP'] = df['MATCHUP'].str.replace('NJN', 'BKN')
    df['MATCHUP'] = df['MATCHUP'].str.replace('CHH', 'CHA')
    df['MATCHUP'] = df['MATCHUP'].str.replace('VAN', 'MEM')
    df['MATCHUP'] = df['MATCHUP'].str.replace('NOH', 'NOP')
    df['MATCHUP'] = df['MATCHUP'].str.replace('NOK', 'NOP')
    df['MATCHUP'] = df['MATCHUP'].str.replace('SEA', 'OKC')


    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
    
    df['HOME_GAME'] = df['MATCHUP'].str.contains('vs').astype(int)
    df = df.dropna(subset = ['E_OFF_RATING'])
    
    return df

In [34]:
clean_data = clean_team_data(team_data)
clean_data


,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TOV,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,E_TM_TOV_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,PCT_FGA_2PT,PCT_FGA_3PT,PCT_PTS_2PT,PCT_PTS_2PT_MR,PCT_PTS_3PT,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM,HOME_GAME
0,2000-01,1610612763,MEM,Vancouver Grizzlies,20001188,2001-04-18,MEM @ GSW,1,48.0,37,92,0.402,4,9,0.444,17,21,0.810,27,31,58,18,14.0,9,10,10,21,0,95,14.0,107.7,106.7,84.9,91.0,22.8,15.7,0.486,1.29,13.5,NaN,0.453,NaN,15.866,15.7,0.424,0.469,1.0,0.196,91.84,89.00,74.17,89.0,0.657,0.902,0.098,0.695,0.379,0.126,0.074,0.179,0.200,0.316,0.485,0.515,0.500,0.500,0.486,0.514,0
1,2000-01,1610612761,TOR,Toronto Raptors,20001182,2001-04-18,TOR @ WAS,1,48.0,43,95,0.453,6,18,0.333,6,8,0.750,15,36,51,28,11.0,9,4,4,17,0,98,6.0,103.7,103.2,92.5,96.8,11.2,6.3,0.651,2.55,20.4,0.321,0.740,0.524,11.638,11.6,0.484,0.497,1.0,0.201,96.98,95.00,79.17,95.0,0.602,0.811,0.189,0.755,0.163,0.184,0.122,0.061,0.133,0.592,0.649,0.351,0.667,0.333,0.651,0.349,0
2,2000-01,1610612756,PHX,Phoenix Suns,20001187,2001-04-18,PHX vs. UTA,1,48.0,35,66,0.530,1,5,0.200,27,39,0.692,7,25,32,21,18.0,8,3,2,23,0,98,5.0,104.1,105.4,98.6,101.1,5.5,4.3,0.600,1.17,17.2,0.333,0.650,0.500,19.116,19.4,0.538,0.589,1.0,0.200,94.24,92.50,77.08,93.0,0.547,0.924,0.076,0.694,0.469,0.031,0.102,0.276,0.194,0.224,0.588,0.412,1.000,0.000,0.600,0.400,1
3,2000-01,1610612742,DAL,Dallas Mavericks,20001185,2001-04-18,DAL vs. MIN,1,48.0,50,91,0.549,5,17,0.294,15,20,0.750,6,39,45,33,14.0,8,9,2,23,2,120,20.0,111.3,114.3,93.9,94.3,17.4,19.9,0.660,2.36,22.5,NaN,0.700,NaN,12.987,13.3,0.577,0.601,1.0,0.199,107.12,105.50,87.92,105.0,0.621,0.813,0.187,0.750,0.350,0.125,0.217,0.125,0.183,0.400,0.622,0.378,1.000,0.000,0.660,0.340,1
4,2000-01,1610612750,MIN,Minnesota Timberwolves,20001185,2001-04-18,MIN @ DAL,0,48.0,39,95,0.411,5,11,0.455,17,26,0.654,14,34,48,23,14.0,7,2,9,17,2,100,-20.0,93.9,94.3,111.3,114.3,-17.4,-19.9,0.590,1.64,16.0,0.300,NaN,NaN,13.153,13.2,0.437,0.470,1.0,0.198,107.12,105.50,87.92,106.0,0.379,0.884,0.116,0.680,0.200,0.150,0.220,0.170,0.120,0.480,0.588,0.412,0.600,0.400,0.590,0.410,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53671,2020-21,1610612748,MIA,Miami Heat,42000121,2021-05-22,MIA @ MIL,0,53.0,36,99,0.364,20,50,0.400,15,19,0.789,12,39,51,22,17.0,8,3,1,23,18,107,-2.0,95.2,97.3,96.9,97.3,-1.6,0.0,0.611,1.29,15.0,0.242,0.683,0.452,15.130,15.5,0.465,0.498,1.0,0.199,101.83,100.53,83.77,110.0,0.449,0.495,0.505,0.299,0.075,0.561,0.019,0.140,0.159,0.224,0.375,0.625,0.800,0.200,0.611,0.389,0
53672,2020-21,1610612738,BOS,Boston Celtics,42000111,2021-05-22,BOS @ BKN,0,48.0,31,84,0.369,11,30,0.367,20,26,0.769,11,29,40,19,12.0,6,12,7,18,22,93,-11.0,96.4,100.0,110.9,111.8,-14.5,-11.8,0.613,1.58,15.0,0.291,0.647,0.462,12.443,12.9,0.435,0.487,1.0,0.199,95.10,93.00,77.50,93.0,0.434,0.643,0.357,0.430,0.065,0.355,0.129,0.215,0.097,0.366,0.500,0.500,0.818,0.182,0.613,0.387,0
53673,2020-21,1610612749,MIL,Milwaukee Bucks,42000121,2021-05-22,MIL vs. MIA,1,53.0,42,96,0.438,5,31,0.161,20,33,0.606,15,49,64,19,17.0,11,1,3,18,23,109,2.0,96.9,97.3,95.2,97.3,1.6,0.0,0.452,1.12,13.0,0.317,0.758,0.548,15.108,15.2,0.464,0.493,1.0,0.199,101.83,100.53,83.77,112.0,0.551,0.677,0.323,0.679,0.165,0.138,0.110,0.183,0.138,0.514,0.459,0.541,0.400,0.600,0.452,0.548,1
53674,2020-21,1610612757,POR,Portland Trail Blazers,42000161,2021-05-22,POR @ DEN,1,48.0,43,91,0.473,19,40,0.475,18,19,0.947,10,29,39,29,

In [48]:
clean_data.isnull().sum().sum()

24640

In [15]:
clean_data.to_csv("../data/clean_data/clean_team_stats.csv")

In [74]:
connection = sqlite3.connect("../data/nba_stats.db")


In [76]:
pd.read_sql_query("SELECT * FROM team_game_stats", connection)

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TOV,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,E_TM_TOV_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,PCT_FGA_2PT,PCT_FGA_3PT,PCT_PTS_2PT,PCT_PTS_2PT_MR,PCT_PTS_3PT,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM,HOME_GAME
0,2000-01,1610612763,MEM,Vancouver Grizzlies,20001188,2001-04-18 00:00:00,MEM @ GSW,1,48.0,37,92,0.402,4,9,0.444,17,21,0.810,27,31,58,18,14.0,9,10,10,21,0,95,14.0,107.7,106.7,84.9,91.0,22.8,15.7,0.486,1.29,13.5,NaN,0.453,NaN,15.866,15.7,0.424,0.469,1.0,0.196,91.84,89.00,74.17,89.0,0.657,0.902,0.098,0.695,0.379,0.126,0.074,0.179,0.200,0.316,0.485,0.515,0.500,0.500,0.486,0.514,0
1,2000-01,1610612761,TOR,Toronto Raptors,20001182,2001-04-18 00:00:00,TOR @ WAS,1,48.0,43,95,0.453,6,18,0.333,6,8,0.750,15,36,51,28,11.0,9,4,4,17,0,98,6.0,103.7,103.2,92.5,96.8,11.2,6.3,0.651,2.55,20.4,0.321,0.740,0.524,11.638,11.6,0.484,0.497,1.0,0.201,96.98,95.00,79.17,95.0,0.602,0.811,0.189,0.755,0.163,0.184,0.122,0.061,0.133,0.592,0.649,0.351,0.667,0.333,0.651,0.349,0
2,2000-01,1610612756,PHX,Phoenix Suns,20001187,2001-04-18 00:00:00,PHX vs. UTA,1,48.0,35,66,0.530,1,5,0.200,27,39,0.692,7,25,32,21,18.0,8,3,2,23,0,98,5.0,104.1,105.4,98.6,101.1,5.5,4.3,0.600,1.17,17.2,0.333,0.650,0.500,19.116,19.4,0.538,0.589,1.0,0.200,94.24,92.50,77.08,93.0,0.547,0.924,0.076,0.694,0.469,0.031,0.102,0.276,0.194,0.224,0.588,0.412,1.000,0.000,0.600,0.400,1
3,2000-01,1610612742,DAL,Dallas Mavericks,20001185,2001-04-18 00:00:00,DAL vs. MIN,1,48.0,50,91,0.549,5,17,0.294,15,20,0.750,6,39,45,33,14.0,8,9,2,23,2,120,20.0,111.3,114.3,93.9,94.3,17.4,19.9,0.660,2.36,22.5,NaN,0.700,NaN,12.987,13.3,0.577,0.601,1.0,0.199,107.12,105.50,87.92,105.0,0.621,0.813,0.187,0.750,0.350,0.125,0.217,0.125,0.183,0.400,0.622,0.378,1.000,0.000,0.660,0.340,1
4,2000-01,1610612750,MIN,Minnesota Timberwolves,20001185,2001-04-18 00:00:00,MIN @ DAL,0,48.0,39,95,0.411,5,11,0.455,17,26,0.654,14,34,48,23,14.0,7,2,9,17,2,100,-20.0,93.9,94.3,111.3,114.3,-17.4,-19.9,0.590,1.64,16.0,0.300,NaN,NaN,13.153,13.2,0.437,0.470,1.0,0.198,107.12,105.50,87.92,106.0,0.379,0.884,0.116,0.680,0.200,0.150,0.220,0.170,0.120,0.480,0.588,0.412,0.600,0.400,0.590,0.410,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53651,2020-21,1610612748,MIA,Miami Heat,42000121,2021-05-22 00:00:00,MIA @ MIL,0,53.0,36,99,0.364,20,50,0.400,15,19,0.789,12,39,51,22,17.0,8,3,1,23,18,107,-2.0,95.2,97.3,96.9,97.3,-1.6,0.0,0.611,1.29,15.0,0.242,0.683,0.452,15.130,15.5,0.465,0.498,1.0,0.199,101.83,100.53,83.77,110.0,0.449,0.495,0.505,0.299,0.075,0.561,0.019,0.140,0.159,0.224,0.375,0.625,0.800,0.200,0.611,0.389,0
53652,2020-21,1610612738,BOS,Boston Celtics,42000111,2021-05-22 00:00:00,BOS @ BKN,0,48.0,31,84,0.369,11,30,0.367,20,26,0.769,11,29,40,19,12.0,6,12,7,18,22,93,-11.0,96.4,100.0,110.9,111.8,-14.5,-11.8,0.613,1.58,15.0,0.291,0.647,0.462,12.443,12.9,0.435,0.487,1.0,0.199,95.10,93.00,77.50,93.0,0.434,0.643,0.357,0.430,0.065,0.355,0.129,0.215,0.097,0.366,0.500,0.500,0.818,0.182,0.613,0.387,0
53653,2020-21,1610612749,MIL,Milwaukee Bucks,42000121,2021-05-22 00:00:00,MIL vs. MIA,1,53.0,42,96,0.438,5,31,0.161,20,33,0.606,15,49,64,19,17.0,11,1,3,18,23,109,2.0,96.9,97.3,95.2,97.3,1.6,0.0,0.452,1.12,13.0,0.317,0.758,0.548,15.108,15.2,0.464,0.493,1.0,0.199,101.83,100.53,83.77,112.0,0.551,0.677,0.323,0.679,0.165,0.138,0.110,0.183,0.138,0.514,0.459,0.541,0.400,0.600,0.452,0.548,1
53654,2020-21,1610612757,POR,Portland Trail Blazers,42000161,2021

In [77]:
connection.close()

In [56]:
df = pd.read_sql_query("SELECT * FROM team_game_stats", connection)

df

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TOV,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,E_TM_TOV_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,PCT_FGA_2PT,PCT_FGA_3PT,PCT_PTS_2PT,PCT_PTS_2PT_MR,PCT_PTS_3PT,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM,HOME_GAME
0,2000-01,1610612763,MEM,Vancouver Grizzlies,20001188,2001-04-18 00:00:00,MEM @ GSW,1,48.0,37,92,0.402,4,9,0.444,17,21,0.810,27,31,58,18,14.0,9,10,10,21,0,95,14.0,107.7,106.7,84.9,91.0,22.8,15.7,0.486,1.29,13.5,NaN,0.453,NaN,15.866,15.7,0.424,0.469,1.0,0.196,91.84,89.00,74.17,89.0,0.657,0.902,0.098,0.695,0.379,0.126,0.074,0.179,0.200,0.316,0.485,0.515,0.500,0.500,0.486,0.514,0
1,2000-01,1610612761,TOR,Toronto Raptors,20001182,2001-04-18 00:00:00,TOR @ WAS,1,48.0,43,95,0.453,6,18,0.333,6,8,0.750,15,36,51,28,11.0,9,4,4,17,0,98,6.0,103.7,103.2,92.5,96.8,11.2,6.3,0.651,2.55,20.4,0.321,0.740,0.524,11.638,11.6,0.484,0.497,1.0,0.201,96.98,95.00,79.17,95.0,0.602,0.811,0.189,0.755,0.163,0.184,0.122,0.061,0.133,0.592,0.649,0.351,0.667,0.333,0.651,0.349,0
2,2000-01,1610612756,PHX,Phoenix Suns,20001187,2001-04-18 00:00:00,PHX vs. UTA,1,48.0,35,66,0.530,1,5,0.200,27,39,0.692,7,25,32,21,18.0,8,3,2,23,0,98,5.0,104.1,105.4,98.6,101.1,5.5,4.3,0.600,1.17,17.2,0.333,0.650,0.500,19.116,19.4,0.538,0.589,1.0,0.200,94.24,92.50,77.08,93.0,0.547,0.924,0.076,0.694,0.469,0.031,0.102,0.276,0.194,0.224,0.588,0.412,1.000,0.000,0.600,0.400,1
3,2000-01,1610612742,DAL,Dallas Mavericks,20001185,2001-04-18 00:00:00,DAL vs. MIN,1,48.0,50,91,0.549,5,17,0.294,15,20,0.750,6,39,45,33,14.0,8,9,2,23,2,120,20.0,111.3,114.3,93.9,94.3,17.4,19.9,0.660,2.36,22.5,NaN,0.700,NaN,12.987,13.3,0.577,0.601,1.0,0.199,107.12,105.50,87.92,105.0,0.621,0.813,0.187,0.750,0.350,0.125,0.217,0.125,0.183,0.400,0.622,0.378,1.000,0.000,0.660,0.340,1
4,2000-01,1610612750,MIN,Minnesota Timberwolves,20001185,2001-04-18 00:00:00,MIN @ DAL,0,48.0,39,95,0.411,5,11,0.455,17,26,0.654,14,34,48,23,14.0,7,2,9,17,2,100,-20.0,93.9,94.3,111.3,114.3,-17.4,-19.9,0.590,1.64,16.0,0.300,NaN,NaN,13.153,13.2,0.437,0.470,1.0,0.198,107.12,105.50,87.92,106.0,0.379,0.884,0.116,0.680,0.200,0.150,0.220,0.170,0.120,0.480,0.588,0.412,0.600,0.400,0.590,0.410,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53651,2020-21,1610612748,MIA,Miami Heat,42000121,2021-05-22 00:00:00,MIA @ MIL,0,53.0,36,99,0.364,20,50,0.400,15,19,0.789,12,39,51,22,17.0,8,3,1,23,18,107,-2.0,95.2,97.3,96.9,97.3,-1.6,0.0,0.611,1.29,15.0,0.242,0.683,0.452,15.130,15.5,0.465,0.498,1.0,0.199,101.83,100.53,83.77,110.0,0.449,0.495,0.505,0.299,0.075,0.561,0.019,0.140,0.159,0.224,0.375,0.625,0.800,0.200,0.611,0.389,0
53652,2020-21,1610612738,BOS,Boston Celtics,42000111,2021-05-22 00:00:00,BOS @ BKN,0,48.0,31,84,0.369,11,30,0.367,20,26,0.769,11,29,40,19,12.0,6,12,7,18,22,93,-11.0,96.4,100.0,110.9,111.8,-14.5,-11.8,0.613,1.58,15.0,0.291,0.647,0.462,12.443,12.9,0.435,0.487,1.0,0.199,95.10,93.00,77.50,93.0,0.434,0.643,0.357,0.430,0.065,0.355,0.129,0.215,0.097,0.366,0.500,0.500,0.818,0.182,0.613,0.387,0
53653,2020-21,1610612749,MIL,Milwaukee Bucks,42000121,2021-05-22 00:00:00,MIL vs. MIA,1,53.0,42,96,0.438,5,31,0.161,20,33,0.606,15,49,64,19,17.0,11,1,3,18,23,109,2.0,96.9,97.3,95.2,97.3,1.6,0.0,0.452,1.12,13.0,0.317,0.758,0.548,15.108,15.2,0.464,0.493,1.0,0.199,101.83,100.53,83.77,112.0,0.551,0.677,0.323,0.679,0.165,0.138,0.110,0.183,0.138,0.514,0.459,0.541,0.400,0.600,0.452,0.548,1
53654,2020-21,1610612757,POR,Portland Trail Blazers,42000161,2021

In [73]:
connection.close()

In [44]:
def add_team_game_stats(conn, if_exists='append'):
    table_name = 'team_game_stats'
    
    if_exists == 'replace':
        conn.execute('DROP TABLE IF EXISTS ' + table_name)

cur.execute("""
    CREATE TABLE nba_team_stats(
    id integer PRIMARY KEY,
    SEASON_YEAR TEXT, 
    TEAM_ID INTEGER, 
    TEAM_ABBREVIATION TEXT, 
    TEAM_NAME TEXT,
    GAME_ID INTEGER,
    GAME_DATE DATE,
    MATCHUP TEXT,
    WL FLOAT,
    MIN FLOAT,
    FGM FLOAT,
    FGA FLOAT,
    FG_PCT FLOAT,
    FG3M FLOAT,
    FG3A FLOAT,
    FG3_PCT FLOAT, 
    FTM FLOAT, 
    FTA FLOAT,
    FT_PCT FLOAT,
    OREB FLOAT,
    DREB FLOAT, 
    REB FLOAT,
    AST FLOAT,
    TOV FLOAT, 
    STL FLOAT,
    BLK FLOAT,
    BLKA FLOAT,
    PF FLOAT, 
    PFD FLOAT,
    PTS FLOAT,
    PLUS_MINUS FLOAT,
    E_OFF_RATING FLOAT,
    OFF_RATING FLOAT,
    E_DEF_RATING FLOAT,
    DEF_RATING FLOAT,
    E_NET_RATING FLOAT, 
    NET_RATING FLOAT,
    AST_PCT FLOAT,
    AST_TOV FLOAT, 
    AST_RATIO FLOAT,
    OREB_PCT FLOAT,
    DREB_PCT FLOAT, 
    REB_PCT FLOAT,
    E_TM_TOV_PCT FLOAT,
    TM_TOV_PCT FLOAT,
    EFG_PCT FLOAT, 
    TS_PCT FLOAT, 
    USG_PCT FLOAT,
    E_USG_PCT FLOAT, 
    E_PACE FLOAT,
    PACE FLOAT,
    PACE_PER40 FLOAT,
    POSS FLOAT,
    PIE FLOAT, 
    PCT_FGA_2PT FLOAT,
    PCT_FGA_3PT FLOAT,
    PCT_PTS_2PT FLOAT, 
    PCT_PTS_2PT_MR FLOAT,
    PCT_PTS_3PT FLOAT,
    PCT_PTS_FB FLOAT,
    PCT_PTS_FT FLOAT, 
    PCT_PTS_OFF_TOV FLOAT,
    PCT_PTS_PAINT FLOAT,
    PCT_AST_2PM FLOAT,
    PCT_UAST_2PM FLOAT,
    PCT_AST_3PM FLOAT, 
    PCT_UAST_3PM FLOAT, 
    PCT_AST_FGM FLOAT,
    PCT_UAST_FGM FLOAT, 
    HOME_GAME FLOAT)""")

conn.commit()

In [58]:
from src.data.make_betting_dataset import clean_spread_data

spread_df = pd.read_csv('../data/betting_data/all_spreads_sbr.csv', parse_dates=['game_date'])
clean_spread = clean_spread_data(spread_df)
clean_spread
clean_spread[['away_team', 'home_team', 'game_date', 'home_spread_mode', 'away_spread_mode']]


from src.data.make_betting_dataset import clean_moneyline_data

ml_df = pd.read_csv('../data/betting_data/all_moneylines_sbr.csv', parse_dates=['game_date'])
clean_moneylines = clean_moneyline_data(ml_df)



In [62]:
clean_spread[['home_spread_mode', 'away_spread_mode']].isnull().sum()

home_spread_mode    116
away_spread_mode    116
dtype: int64